# Navigator Agent: Custom Subagent Extension

This notebook demonstrates how to create and integrate a custom subagent into the Navigator Agent workflow.

In [ ]:
from typing import Dict, Any, List
from pydantic import BaseModel, Field
from langchain_core.language_models import BaseLanguageModel
from langchain_openai import ChatOpenAI

# Import base subagent and state schemas
from src.navigator_agent.agents.subagents.base import BaseSubagent
from src.navigator_agent.schemas.agent_state import Solution, SolutionStatus

In [ ]:
class SecurityComplianceSubagent(BaseSubagent):
    """Custom subagent for security and compliance analysis."""
    
    def __init__(self, llm: BaseLanguageModel):
        super().__init__(llm)
        self.name = "security_compliance_agent"
    
    def analyze_security(self, solutions: List[Solution]) -> List[Solution]:
        """
        Enhance solutions with security and compliance insights.
        
        Args:
            solutions: List of potential solutions
        
        Returns:
            Enhanced solutions with security analysis
        """
        enhanced_solutions = []
        
        for solution in solutions:
            # Use LLM to analyze security aspects
            security_prompt = f"""Analyze the security implications of this solution:
{solution.content}

Provide a detailed security assessment including:
1. Potential vulnerabilities
2. Compliance risks
3. Recommended security improvements
"""
            
            security_response = self.llm.invoke(security_prompt)
            
            # Update solution with security metrics
            solution.evaluation_metrics['security_score'] = self._calculate_security_score(
                solution, security_response.content
            )
            solution.metadata['security_analysis'] = security_response.content
            
            enhanced_solutions.append(solution)
        
        return enhanced_solutions
    
    def _calculate_security_score(self, solution: Solution, analysis: str) -> float:
        """
        Calculate a security score based on the analysis.
        
        Args:
            solution: Original solution
            analysis: Security analysis text
        
        Returns:
            Normalized security score
        """
        # Simple heuristic for security scoring
        risk_keywords = ['vulnerability', 'risk', 'threat', 'exploit']
        security_keywords = ['encryption', 'authentication', 'secure', 'protection']
        
        risk_count = sum(analysis.lower().count(keyword) for keyword in risk_keywords)
        security_count = sum(analysis.lower().count(keyword) for keyword in security_keywords)
        
        # Normalize score between 0 and 1
        security_score = max(0, 1 - (risk_count * 0.2 - security_count * 0.1))
        return round(security_score, 2)

In [ ]:
from src.navigator_agent.agents.navigator import NavigatorAgent

# Initialize components
llm = ChatOpenAI(model="gpt-4", temperature=0.5)

# Create custom security compliance subagent
security_agent = SecurityComplianceSubagent(llm)

# Initialize Navigator Agent
navigator = NavigatorAgent(llm)

# Inject custom subagent into workflow
navigator.register_subagent(security_agent)

In [ ]:
# Define a problem with security focus
problem_statement = "Design a secure cloud-native banking application with multi-factor authentication"

# Process problem with custom security analysis
workflow_result = navigator.process_input(problem_statement)

# Display solutions with security insights
print("Solutions with Security Analysis:")
for solution in workflow_result.possible_solutions:
    print(f"\nSolution ID: {solution.id}")
    print(f"Security Score: {solution.evaluation_metrics.get('security_score', 'N/A')}")
    print("Security Analysis:")
    print(solution.metadata.get('security_analysis', 'No detailed analysis'))